# Import Data

In [63]:
import os
import pandas as pd
import numpy as np
import scipy.optimize as sc 

In [60]:
# Import data from files
os.chdir('/Users/talhajamal/Desktop/Code/BigData1Project')
data = pd.read_csv("data/Returns_Data.csv")
data['date'] = pd.to_datetime(data['date'], dayfirst=False)
characteristics = pd.read_csv("data/Stock_Characteristics_Data.csv")
dictionary = pd.read_excel("data/StockDataDictionary.xlsx")

# Create new dataframes
prices = data.pivot(index='date', columns='ticker', values='PRC')
volume = data.pivot(index='date', columns='ticker', values='VOL')
returns = data.pivot(index='date', columns='ticker', values='RET')
#returns = returns * 100 # Scale returns to percentage
# Summary of Returns
returns_summary = returns.describe()
shares_outstanding = data.pivot(index='date', columns='ticker', values='SHROUT')
value_weighted_returns = data.pivot(index='date', columns='ticker', values='vwretd')
equal_weighted_returns = data.pivot(index='date', columns='ticker', values='ewretd')
tickers = prices.columns # List of Tickers

# Functions

In [115]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    """ Calculate Portfolio Performance"""
    annualizedReturns = np.sum(meanReturns*weights)*252
    annualizedStd = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(252)
    return annualizedReturns*100, annualizedStd*100

def negativeSR(weights, meanReturns, covMatrix,riskFreeRate = 0):
    """ Calculate Negative Sharpe so that Optimizer works on it and minimizes it -> essentially maximising it"""
    annualizedRet, annualizedStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - ((annualizedRet - riskFreeRate)/annualizedStd)

def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0, 1)):
    """ Minimize the negative Sharpe Ratio -> Maximize it"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x) - 1 }) # Weights must sum up to 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets)) # For every asset have this bound
    result = sc.minimize(negativeSR, x0=numAssets*[1./numAssets], args=args, 
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0, 1)):
    """ Minimize the portfolio variance by altering the weights/allocations of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x) - 1 }) # Weights must sum up to 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets)) # For every asset have this bound
    result = sc.minimize(portfolioVariance, x0=numAssets*[1./numAssets], args=args, 
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# Test Functions

In [113]:
eqWeights = np.array([1/100 for x in returns])
meanReturns = returns.mean()
covMatrix = returns.cov()
equalWeightedRet, equalWeightedVol = portfolioPerformance(eqWeights, meanReturns, covMatrix)
print("Equal Weighted Portfolio Returns: ", equalWeightedRet, "%")
print("Equal Weighted Portfolio Volatility: ", equalWeightedVol, "%")
print("Sharpe", negativeSR(eqWeights, meanReturns, covMatrix))
eqWeightedPortfolioOptimization = maxSR(meanReturns, covMatrix, riskFreeRate=0)
eqWeightMaxSR = round(eqWeightedPortfolioOptimization.fun * (-1), 4)
print("Optimized Equal Weighted Portfolio Sharpe:", eqWeightMaxSR)
eqWeightedPortfolioWeights = eqWeightedPortfolioOptimization.x

Equal Weighted Portfolio Returns:  15.931604359696639 %
Equal Weighted Portfolio Volatility:  18.619823475582756 %
Sharpe -0.8556259612551466
Optimized Equal Weighted Portfolio Sharpe: 1.4624


In [114]:
optimizedEqualWeightedRet, optimizedEqualWeightedVol = portfolioPerformance(eqWeightedPortfolioWeights, meanReturns, covMatrix)
print("Optimized Equal Weighted Portfolio Returns: ", optimizedEqualWeightedRet, "%")
print("Optimized Equal Weighted Portfolio Volatility: ", optimizedEqualWeightedVol, "%")

Optimized Equal Weighted Portfolio Returns:  22.681563109201992 %
Optimized Equal Weighted Portfolio Volatility:  15.509767741809984 %
